In [ ]:
import os

arxiv_output_dir = './arxiv'

In [ ]:
date_folders = os.listdir(arxiv_output_dir)
date_folders.sort()

if '.DS_Store' in date_folders:
    date_folders.remove('.DS_Store')

date_folders

In [ ]:
from tqdm.notebook import tqdm

import os
import gzip
import tarfile
import shutil

from joblib import Parallel, delayed
from joblib_progress import joblib_progress

def extract_mixed_gz(gz_path, output_folder):
    """
    Extract a .gz file which could either contain a single file
    or multiple files (e.g., in the form of a .tar).

    Args:
    - gz_path: Path to the .gz file.
    - output_folder: Folder where the .gz file should be extracted to.

    Returns:
    None.
    """
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # First, we decompress the gz file
    decompressed_file_path = os.path.join(output_folder, os.path.basename(gz_path).replace('.gz', '') + '-temp')
    with gzip.open(gz_path, 'rb') as f_in:
        with open(decompressed_file_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    extraction_folder = os.path.join(output_folder, os.path.basename(gz_path).replace('.gz', ''))
    if not os.path.exists(extraction_folder):
        os.makedirs(extraction_folder)

    # Check if the decompressed result is a .tar file
    if tarfile.is_tarfile(decompressed_file_path):
        # Extract the tar file contents
        with tarfile.open(decompressed_file_path, 'r') as tar:
            tar.extractall(path=extraction_folder)
        # Remove the temporary decompressed tar file
        os.remove(decompressed_file_path)
    else:
        shutil.move(decompressed_file_path, os.path.join(extraction_folder, os.path.basename(gz_path).replace('.gz', '') + '.tex'))

def extract_gz_to_tex(date_dir):
    tex_output_dir = os.path.join(arxiv_output_dir, date_dir, 'tex')
    os.makedirs(os.path.join(arxiv_output_dir, date_dir, 'src_invalid'), exist_ok=True)
    pending_files = os.listdir(os.path.join(arxiv_output_dir, date_dir, 'src'))
    pending_files.sort()

    def inner_func(filename):
        try:
            src_path = os.path.join(arxiv_output_dir, date_dir, 'src', filename)
            if not os.path.exists(src_path) or os.path.getsize(src_path) == 0:
                return None
            filename_stem = os.path.splitext(filename)[0]
            filename_ext = os.path.splitext(filename)[1]
            if filename_ext == '.pdf':
                shutil.move(src_path, os.path.join(arxiv_output_dir, date_dir, 'src_invalid', filename))
            elif filename_ext == '.gz':
                extract_mixed_gz(src_path, tex_output_dir)
            elif filename_ext == '.tex':
                shutil.move(src_path, os.path.join(arxiv_output_dir, date_dir, 'tex', filename))
        except Exception as e:
            print('[ERROR]', str(e))
        return filename
    
    with joblib_progress("Preprocessing...", total=len(pending_files)):
        Parallel(n_jobs=os.cpu_count(), pre_dispatch='1*n_jobs')(
            delayed(inner_func)(f) for f in pending_files
        )

In [ ]:
processing_dates = [
    # '0001',
    # '0002',
    # '0003',
    # '0004',
    # '0005',
    # '0006',
    # '0007',
    # '0008',
    # '0009',
    # '0010',
    # '0011',
    # '0012',
    # '0501',
    # '1001',
    # '1501',
    # '2001',
]

In [ ]:
for date_folder_name in processing_dates:
    extract_gz_to_tex(date_folder_name)
    print(date_folder_name, len(os.listdir(os.path.join(arxiv_output_dir, date_folder_name, 'tex'))))

# LaTeXML

In [ ]:
import os
import subprocess
import random  # Temp.
from joblib import Parallel, delayed
from joblib_progress import joblib_progress

def convert_tex_to_html(datedir):
    html_output_dir = os.path.join(arxiv_output_dir, datedir, 'html')
    if not os.path.exists(html_output_dir):
        os.mkdir(html_output_dir)
    
    log_output_dir = os.path.join(arxiv_output_dir, datedir, 'html_log')
    if not os.path.exists(log_output_dir):
        os.mkdir(log_output_dir)
    
    def inner_func(dirname):
        dirpath = os.path.join(arxiv_output_dir, datedir, 'tex', dirname)
        sub_files = os.listdir(dirpath)
        tex_files = [file for file in sub_files if file.lower().endswith('.tex')]
        if len(tex_files) == 0:
            return 'Not found'
        first_tex_filepath = os.path.join(dirpath, tex_files[0])
        output_path = os.path.join(html_output_dir, dirname + '.html')
        log_path = os.path.join(log_output_dir, dirname + '.log')
    
        if not os.path.exists(output_path):
            command = ['latexmlc', '--dest=' + output_path, '--log=' + log_path, first_tex_filepath]
            result = subprocess.run(' '.join(command), shell=True, stderr=subprocess.DEVNULL)

        if not os.path.exists(output_path):
            return 'Failed in LaTeXML'
    
        # If the output file is less than 5 KB, then we remove it.
        if os.path.getsize(output_path) < 4000:
            os.remove(output_path)
            return 'Invalid size'
        return 'OK'
    
    print("CPU:", os.cpu_count())
    pending_tex_files = os.listdir(os.path.join(arxiv_output_dir, datedir, 'tex'))
    pending_tex_files = random.sample(pending_tex_files, 800)
    pending_tex_files.sort()
    
    with joblib_progress("Processing...", total=len(pending_tex_files)):
        result = Parallel(n_jobs=os.cpu_count(), pre_dispatch='1*n_jobs')(
            delayed(inner_func)(dirname) for dirname in pending_tex_files
        )
        
    print('Done!')
    return result

In [ ]:
def preprocess(date_folder_name):
    res_curr_date = convert_tex_to_html(date_folder_name)
    print(date_folder_name, sum([i == 'OK' for i in res_curr_date]), sum([i != 'OK' for i in res_curr_date]))

In [ ]:
# preprocess('0001')

In [ ]:
# preprocess('0002')

In [ ]:
# preprocess('0003')

In [ ]:
# preprocess('0004')

In [ ]:
# preprocess('0005')

In [ ]:
# preprocess('0006')

In [ ]:
# preprocess('0007')

In [ ]:
# preprocess('0008')

In [ ]:
# preprocess('0009')

In [ ]:
# preprocess('0010')

In [ ]:
# preprocess('0011')

In [ ]:
# preprocess('0012')

In [ ]:
# preprocess('0501')

In [ ]:
# preprocess('1001')

In [ ]:
# preprocess('1501')

In [ ]:
# preprocess('2001')

In [ ]:
# This block is used to clean up unnecessary logs in the working dir.

# import glob
# import os

# # Find all .log files in the current directory
# log_files = glob.glob('./*.log')

# # Remove each file
# for file in log_files:
#     os.remove(file)
#     print(f"Removed {file}")